In [1]:
import pandas as pd
import re
import nltk
import numpy as np
#nltk.download()
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [2]:
newsData = pd.read_csv("Combined_News_DJIA.csv")
stockData = pd.read_csv("DJIA_table.csv")
stockData = stockData.iloc[::-1]

In [3]:
newsTrain = newsData[newsData['Date']< '2015-01-01']
newsTest = newsData[newsData['Date']>'2014-12-31']
stockTest = stockData[stockData['Date']>'2014-12-31']
stockTest.head(2)

,Date,Open,High,Low,Close,Volume,Adj Close
377,2015-01-02,17823.070312,17951.779297,17731.300781,17832.990234,76270000,17832.990234
376,2015-01-05,17821.300781,17821.300781,17475.929688,17501.650391,116160000,17501.650391


In [4]:
def news_to_words( news ):
    # 1. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", news) 
    #
    # 2. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 3. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 4. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 5. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))

In [5]:
trainheadlines = []
for row in range(0,len(newsTrain.index)):
#for row in range(0,2):
    #pre-process words
    combinedNews = " ".join(str(x).replace("b\"","").replace("b\'","") for x in newsTrain.iloc[row,2:27])
    processedNews = news_to_words(combinedNews)
    trainheadlines.append(processedNews)
    #trainheadlines.append(combinedNews)
#print(trainheadlines)
#print(len(newsTrain.index))

In [6]:
#regardless the characters
#maxium feature words we are considering
vectorizer = CountVectorizer(analyzer = "word",   \
                             ngram_range=(2,3),\
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 1000)
newsTrain_feature = vectorizer.fit_transform(trainheadlines)
newsTrain_feature = newsTrain_feature.toarray()
print (newsTrain_feature.shape)

(1611, 1000)


In [7]:
vocab = vectorizer.get_feature_names()
print (vocab)
#print(newsTrain_feature[0:2])

['abc news', 'abuse scandal', 'according new', 'across country', 'across europe', 'across world', 'adolf hitler', 'afghan police', 'afghan president', 'afghan women', 'african republic', 'ai weiwei', 'aid ship', 'air base', 'air force', 'air france', 'air pollution', 'air strike', 'air strikes', 'aircraft carrier', 'al assad', 'al jazeera', 'al jazeera english', 'al qaeda', 'al qaida', 'amnesty international', 'anders behring', 'anders behring breivik', 'angela merkel', 'anti corruption', 'anti gay', 'anti government', 'anti islam', 'anti piracy', 'anti semitism', 'anti terror', 'anyone else', 'arab league', 'arab spring', 'arab world', 'arctic sea', 'armed forces', 'armenian genocide', 'around world', 'arrest warrant', 'associated press', 'asylum seekers', 'attack iran', 'aung san', 'aung san suu', 'austerity measures', 'australian government', 'australian pm', 'australian pm tony', 'australian prime', 'australian prime minister', 'ban ki', 'ban ki moon', 'bank accounts', 'bank americ

In [8]:
# example = vectorizer.build_tokenizer()(' '.join(trainheadlines))
# pd.DataFrame([[x,example.count(x)] for x in set(example)], columns = ['Word', 'Count'])

In [9]:
print ("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 10) #n_estimators = 10 or 100
forest = forest.fit( newsTrain_feature, newsTrain["Label"] )

Training the random forest...


In [10]:
#testing
print ("Testing the testing dataset...")
testheadlines = []
for row in range(0,len(newsTest.index)):
#for row in range(0,2):
    #pre-process words
    combinedtestNews = " ".join(str(x).replace("b\"","").replace("b\'","") for x in newsTest.iloc[row,2:27])
    processedtestNews = news_to_words(combinedtestNews)
    testheadlines.append(processedtestNews)
newsTest_features = vectorizer.transform(testheadlines)
newsTest_features = newsTest_features.toarray()
result = forest.predict(newsTest_features)
output = pd.DataFrame( data={"Date":newsTest["Date"],"Actual Label":newsTest["Label"],"Predict Label":result})
output.to_csv( "Bag_of_Words_rand_forest.csv", index=False, quoting=3 )

Testing the testing dataset...


In [11]:
#df = pd.read_csv("Bag_of_Words_rand_forest.csv")
a = np.where((output['Actual Label'] == output['Predict Label']))
b = np.array(output['Actual Label'])
c = np.array(output['Predict Label'])
count = np.count_nonzero(b == c)
print("accuracy:",count/378)
#print(a)

accuracy: 0.5343915343915344


In [12]:
open_and_close = pd.DataFrame({'Date':stockTest["Date"],'Open':stockTest['Open'],"Close":stockTest["Close"]})
open_and_close.to_csv("open_and_close_test.csv", index= False, quoting =3)